# Installation H2 

Le système utilisé pendant les TME est H2.

In [ ]:
pip install psycopg2-binary

Télécharger le pilote de H2

In [ ]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar -O postgresql-42.6.0.jar
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar -O h2-2.1.214.jar

**Relancez le kernel**: Kernel -> Restart kernel ...

https://www.psycopg.org/docs/
http://localhost:8082

In [ ]:
import psycopg2
import os
local_dir = "./data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

## Démarrer le serveur SQL H2

In [5]:
#Remplacez les valeurs des variables suivantes
#NN= numéro de groupe: chaine de caractères (ex: 01, 02,.., 13), remplacer xx 
NN = 'xx'
#BB=numero binome: chaîne de caractères (assigné par l'enseignant, ex: 01,..., 15), remplacer xx
BB = 'xx'
port = 5000+int(NN+BB)
print(f'Le numero du port utilisé pour la connexion à la BD est: {port}')

Le numero du port utilisé pour la connexion à la BD est: 5110


In [6]:
%%bash --bg --out output -s "$port"
java -Dh2.bindAddress=127.0.0.1 -cp h2-2.1.214.jar:postgresql-42.6.0.jar org.h2.tools.Server -pg -pgPort $1 -baseDir ./data -ifNotExists

# Fonctions utiles

## connect_H2

In [7]:
def connect_H2(db,user,password):
    global connection
    try:
        connection
    except:
        connection = None
    if connection != None:
        try:
            connection.close()
            print("Connection closed")
        except  Error as e:
            print(f"The error '{e}' occurred")
    try:
        # connection = sqlite3.connect(path,isolation_level='DEFERRED')
        connection = psycopg2.connect(f"dbname={db} user={user} password={password} host=localhost port={port}")
        print("Connection to H2 DB successful")
    except Exception as e:
        print(f"The error '{e}' occurred")
    return connection

## execute

In [8]:
def execute(connection, query, show=True,close=True):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        print(cursor.rowcount,"rows")
        if show and cursor.rowcount:
            names = [desc[0] for desc in cursor.description]
            for attr in names:
                print(str(attr[:16]).ljust(16),end='|')
            print()
            for attr in names:
                print(''.ljust(17,'-'),end='')
            print()
            for ligne in cursor:
                for attr in ligne:
                    print(str(attr)[:16].ljust(16),end='|')
                print()
        if close:
            cursor.close()
    except Exception as e:
        print(f"The error '{e}' occurred")
        cursor=None
    return cursor

**TD/TME 6 : REQUÊTES IMBRIQUÉES AVEC EXISTS, ALL ET ANY**

Ce TD/TME utilise les données contenues dans le fichier **bd_jo_v2_H2.sql**

**Attention**: vous pouvez cliquer sur les 3 lignes dans la fenêtre de gauche pour d'afficher les différentes sections du notebook 

# TD6

On considère le schéma de la base JEUXOLYMPIQUE2014 qui décrit les athlètes et leurs résultats aux
épreuves des sports des Jeux Olympiques d'Hiver Sotchi 2014 :

**PAYS** ( <u>CODEPAYS</u>, NOMP)<br/>
Ex. ('FRA', 'France')<br/>
**SPORT** ( <u>SID</u>, NOMSP)<br/>
Ex. (1, 'Biathlon')<br/>
**EPREUVE** ( <u>EPID</u>, SID*, NOMEP, CATÉGORIE, DATEDEBUT, DATEFIN)<br/>
Ex. (10, 1, 'relais 4x7,5km', 'Hommes', 22/02/2014, 22/02/2014)<br/>
**ATHLETE** ( <u>AID</u>, NOMATH, PRENOMATH, DATENAISSANCE, CODEPAYS*)<br/>
Ex. (1000, 'SOBOLEV', 'Alexey', NULL, 'RUS')<br/>
**EQUIPE** ( <u>EQID</u>, CODEPAYS*)<br/>
Ex. (30, 'SUI')<br/>
**ATHLETESEQUIPE** ( <u>EQID*, AID*</u>)<br/>
Ex. (30, 796) : L'athlète (aid=796) a participé à l'équipe (eqid=30)<br/>
**RANGINDIVIDUEL** ( <u>EPID*, AID*</u>, RANG)<br/>
Ex. (15, 61, 1) : L'athlète (aid=61) a gagné la médaille d'or (rang=1) de l'épreuve (epid=15)<br/>
**RANGEQUIPE** ( <u>EPID*, EQID*</u>, RANG)<br/>
Ex. (10, 30, 14) : L'équipe (eqid=30) a été classée 14e à l'épreuve (epid=10)<br/>


Les attributs qui forment la clé primaire de chaque relation sont soulignés. Les clés étrangères sont
signalées avec une *. Les attributs aid, epid, eqid et sid correspondent aux identifiants des athlètes,
épreuves, équipes et sports et sont utilisés à la fois comme clé primaire ou comme référence (clé
étrangère) vers la relation correspondante.
La relation **PAYS** contient le code et le nom de tous les pays, même si ils n'ont pas participé aux
Jeux Olympiques. Les sports (n-uplets de la relation **SPORT**) sont un ensemble d'épreuves (n-uplets
de la relation **EPREUVE**). Pour chaque épreuve on connaît son nom et les date de début et fin de
l'épreuve. Les épreuves peuvent être individuelles ou par équipe. Dans le premier cas, la
participation des athlètes (n-uplets de la relation ATHLETE) est stocké dans la table
**RANGINDIVIDUEL** qui contient en plus le rang qu'ils ont obtenu (1 pour la médaille d'or). Pour les
épreuves par équipe les résultats sont stockés dans la relation **RANGEQUIPE**, alors que l'information
sur le pays de chaque équipe et ses participants et stocké dans les relations **EQUIPE** et
**ATHLETESEQUIPE**. Dans les relations **RANGINDIVIDUEL** et **RANGEQUIPE** l'attribut rang est égal à
null si l'athlète ou l'équipe a été disqualifié.




## Connexion à H2

In [ ]:
connection = connect_H2("jo"+f'{port}',"ba","ba")

## Créer les tables et charger les données bd_jo_v2_H2.sql

In [ ]:
schemafile=open("bd-jo-v2_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
connection.commit()

**Requêtes: ANY /ALL / IN / EXISTS**

## **Q1**. 
Les athlètes ayant participé à (au moins) une épreuve individuelle et (au moins) une épreuve par équipe. Exprimer la requête en utilisant ANY (au lieu de IN; voir requête 2.6 dans TD5).
Résultat : (372 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

 ## **Q2. a)** 
Les pays ayant au moins un athlète (requête équivalente: les pays ayant participé aux JO).
Résultat : (88 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

##  **Q2. b**) 
Les pays ayant exactement un seul athlète
Résultat : (18 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

## **Q3. a)** 
Les athlètes qui n'ont jamais été disqualifiés aux épreuves individuelles.
Résultat : (2194 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

## **Q3. b)** 
Les pays qui n'ont pas eu d'athlète disqualifié aux épreuves individuelles.
Résultat : (143 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

##  **Q3. c)** 
Les pays qui ont participé aux JO et qui n'ont pas eu d'athlètes disqualifiés aux épreuves
individuelles.
Résultat : (25 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

##  **Q3. d)** 
Les pays qui ont participé aux JO et qui n'ont pas eu des athlètes disqualifiés ni en
individuel ni par équipe
Résultat : (25 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

##  **Q4. a)** 
Les athlètes n'ayant pas gagné de médaille ni en individuel ni en équipe
Résultat : (1921 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

##  **Q4. b)** 
Les pays ayant participé aux JO et n'ayant pas gagné de médaille ni en individuel ni en
équipe.
Résultat : (63 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)


##  **Q5.** 
L'(es) épreuve(s) avec la plus grande durée.
Résultat : Hockey sur glace, Hockey sur glace, Hommes, 16 jours

In [ ]:
query="""


"""
execute(connection,query,show=True)

##  **Q6.**  
Les athlètes ayant gagné une médaille à toutes les épreuve individuelles auxquelles ils ont participé. Résultat : (109 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

**<u>BASE FOOFLE</u>**

## Créer les tables et charger les données base_foofle_H2.sql

In [ ]:
schemafile=open("base_foofle_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
connection.commit()

##  **Q7.**  
Quelles équipes ont déjà joué au stade préféré de l'équipe des Piepla ? Résultats : (2 lignes) Direkt , Piepla

In [ ]:
query="""


"""
execute(connection,query,show=True)

##  **Q8.**  
Quels sont les stades où a déjà joué Manon Messi ? Résultats: (3 lignes) GrandArena, Boulodrome, BukHall

In [ ]:
query="""


"""
execute(connection,query,show=True)

##  **Q9.**  
A quelle date a eu lieu un match entre deux équipes sponsorisées par le même sponsor ? Résultats :(4 lignes) 2015-05-16, 2015-05-15, 2015-06-15, 2015-05-12

In [ ]:
query="""


"""
execute(connection,query,show=True)

##  **Q10.**  
Quel sponsor a financé deux joueurs différents ayant eu un match le même jour et dans des
stades différents mais proches (moins de 50 km) ? Résultats :(4 lignes) Air Monaco, Carouf, Robek, Adadis

In [ ]:
query="""
 
 
"""
execute(connection,query,show=True)

# TME6

On considère le schéma de la base JEUXOLYMPIQUE2014 donné en TD :

**PAYS** ( <u>CODEPAYS</u>, NOMP)<br/>
Ex. ('FRA', 'France')<br/>
**SPORT** ( <u>SID</u>, NOMSP)<br/>
Ex. (1, 'Biathlon')<br/>
**EPREUVE** ( <u>EPID</u>, SID*, NOMEP, CATÉGORIE, DATEDEBUT, DATEFIN)<br/>
Ex. (10, 1, 'relais 4x7,5km', 'Hommes', 22/02/2014, 22/02/2014)<br/>
**ATHLETE** ( <u>AID</u>, NOMATH, PRENOMATH, DATENAISSANCE, CODEPAYS*)<br/>
Ex. (1000, 'SOBOLEV', 'Alexey', NULL, 'RUS')<br/>
**EQUIPE** ( <u>EQID</u>, CODEPAYS*)<br/>
Ex. (30, 'SUI')<br/>
**ATHLETESEQUIPE** ( <u>EQID*, AID*</u>)<br/>
Ex. (30, 796) : L'athlète (aid=796) a participé à l'équipe (eqid=30)<br/>
**RANGINDIVIDUEL** ( <u>EPID*, AID*</u>, RANG)<br/>
Ex. (15, 61, 1) : L'athlète (aid=61) a gagné la médaille d'or (rang=1) de l'épreuve (epid=15)<br/>
**RANGEQUIPE** ( <u>EPID*, EQID*</u>, RANG)<br/>
Ex. (10, 30, 14) : L'équipe (eqid=30) a été classée 14e à l'épreuve (epid=10)<br/>

**Requêtes: ANY /ALL / IN / EXISTS**

## **Q1.** 
Les sports dont toutes les épreuves ont duré un seul jour.
Résultat : Ski de fond, Ski alpin, Biathlon

In [ ]:
query="""


"""
execute(connection,query,show=True)

## **Q2.** 
Les sports qui n'ont pas d'épreuves de categorie Mixte.
Résultat : (12 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

## **Q3.** 
Les équipes dont aucun athlète n'a gagné de médaille aux épreuves individuelles.
Attention : il y a des équipes sans athlètes.
Résultat : (265 lignes avec les équipes sans athlètes - 252 lignes sans les équipes sans
athlètes)

Avec les équipes sans athlètes :

In [ ]:
query="""

"""
execute(connection,query,show=True)

Sans les équipes sans athlètes :

In [ ]:
query="""


"""
execute(connection,query,show=True)

## **Q4.** 
La nationalité de l'athlète le/la plus jeune. Attention : il y a des athlètes dont on ne connaît pas la date de naissance.
Résultat : ('29/11/1998','JPN')

In [ ]:
query="""


"""
execute(connection,query,show=True)

## **Q5.** 
Le plus jeune athlète de chaque pays.
Résultat : (26 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

# Fermer la connexion

In [ ]:
connection.commit() # implicit avec close
connection.close()

In [ ]:
connection